In [195]:
import pytz
import datetime
import googlemaps
import numpy as np
import pandas as pd
from itertools import permutations
from scipy.optimize import linprog
from gcsa.google_calendar import GoogleCalendar

In [63]:
gcal = GoogleCalendar(credentials_path='../secrets/google_oauth_credentials.json')

with open('../secrets/google_api_key') as fp:
    gmaps = googlemaps.Client(key=fp.read())

In [64]:
time_min = datetime.datetime.now()
time_max = time_min + datetime.timedelta(days=2)
past = []
for event in gcal.get_events(time_min=time_min, time_max=time_max, query='Open House'):
    if event.location not in past:
        print(f"- {event.location}")
        print(f"\t- {event.description.split('URL: ')[1]}")
        past.append(event.location)

- 34 Everett Ave #1, Somerville, MA 02145
	- https://www.redfin.com/MA/Somerville/34-Everett-Ave-02145/unit-1/home/8760050
- 31 Concord Ave #2, Cambridge, MA 02138
	- https://www.redfin.com/MA/Cambridge/31-Concord-Ave-02138/unit-2/home/11586215
- 22 Ossipee Rd #22, Somerville, MA 02144
	- https://www.redfin.com/MA/Somerville/22-Ossipee-Rd-02144/unit-22/home/180348621
- 12 Saint Paul St #1, Cambridge, MA 02139
	- https://www.redfin.com/MA/Cambridge/12-St-Paul-St-02139/unit-1/home/11600280
- 10 Grant St #1, Somerville, MA 02145
	- https://www.redfin.com/MA/Somerville/10-Grant-St-02145/unit-1/home/170594128
- 10 Gussie Ter, Somerville, MA 02143
	- https://www.redfin.com/MA/Somerville/10-Gussie-Ter-02143/home/8706459


In [138]:
##dropdown
# pytz.common_timezones

In [301]:
zone = 'America/New_York'
tz = pytz.timezone(zone)
epoch = datetime.datetime.fromtimestamp(0, tz)
time_min = datetime.datetime.now(tz)
time_max = time_min + datetime.timedelta(days=1)
home_loc = '43 Westland Ave, Boston, MA 02115'
events = gcal.get_events(time_min=time_min, time_max=time_max, timezone=zone, query='Open House')
events = pd.DataFrame(map(vars, events))[['start', 'end', 'location']]
events = pd.concat([pd.DataFrame([{'location': home_loc}]), events]).reset_index(drop=True)
events[['start', 'end']] = events[['start', 'end']].applymap(
    lambda x: (x - epoch).total_seconds()
)

In [302]:
dmat = gmaps.distance_matrix(events['location'], events['location'], avoid='highways', mode='driving')
dmat_frame = pd.DataFrame(
    [
        [col['duration']['value'] for col in row['elements']]
        for row in dmat['rows']
    ],
    index=events['location'],
    columns=events['location'])

In [353]:
T_buf = 5 * 60  # parking, etc.
end_buf = 10 * 60  # close to end, etc.

opt_res = None
opt_nodes = None
opt_edges = None
opt_events = None

for r in range(len(events) - 1, 0, -1):
    for p in permutations(events['location'][1:], r):
        nodes = [home_loc, *p]
        sort_events = events.set_index('location').loc[nodes].reset_index()

        links = list(zip(nodes[:-1], nodes[1:]))
        edges = [dmat_frame.loc[l] + T_buf for l in links]

        c = [-1, -1]  # t0, d

        A = np.array([np.ones_like(edges), np.arange(len(edges))]).T
        A = np.append(-A, A, axis=0)

        b = np.cumsum(edges)
        b = np.append(-sort_events['start'].iloc[1:] + b, sort_events['end'].iloc[1:] - end_buf - b)

        t0_bounds = (None, None)
        d_bounds = (5 * 60, 15 * 60)

        res = linprog(c, A_ub=A, b_ub=b, bounds=[t0_bounds, d_bounds])

        if res.success:
            if (opt_res is None) or (res.fun < opt_res.fun):
                opt_res = res
                opt_nodes = nodes
                opt_edges = edges
                opt_events = sort_events
    if opt_nodes is not None:
        break

t0, d = opt_res.x

opt_events = opt_events.assign(
    arrive=[np.nan] + [t0 + i * d + sum(opt_edges[:i + 1]) for i in range(len(opt_edges))],
    leave=[t0] + [t0 + (i + 1) * d + sum(opt_edges[:i + 1]) for i in range(len(opt_edges))]
)

opt_events[['start', 'end', 'arrive', 'leave']] = (
    opt_events[['start', 'end', 'arrive', 'leave']].applymap(
        lambda x: x if np.isnan(x) else datetime.datetime.fromtimestamp(x, tz).strftime('%I:%M%p')
    )
)

In [354]:
opt_events

,location,start,end,arrive,leave
0,"43 Westland Ave, Boston, MA 02115",NaN,NaN,NaN,12:05PM
1,"12 Saint Paul St #1, Cambridge, MA 02139",11:00AM,01:00PM,12:23PM,12:32PM
2,"10 Grant St #1, Somerville, MA 02145",11:00AM,01:00PM,12:49PM,12:59PM
3,"34 Everett Ave #1, Somerville, MA 02145",12:00PM,01:30PM,01:10PM,01:19PM
4,"10 Gussie Ter, Somerville, MA 02143",01:30PM,02:30PM,01:34PM,01:43PM
5,"22 Ossipee Rd #22, Somerville, MA 02144",01:30PM,03:00PM,01:56PM,02:06PM
